In [3]:
#import library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [4]:
#data set
dataset = [['Pain','Beurre','Confiture'],
           ['Beurre','Coca'],
           ['Beurre','Lait'],
           ['Pain','Beurre','Coca'],
           ['Pain','Lait'],
           ['Beurre','Lait'],
           ['Pain','Lait'],
           ['Pain','Beurre','Lait','Confiture'],
           ['Pain','Beurre','Lait']
 ]


In [6]:
freq_itemsets = pd.DataFrame(dataset)
print(freq_itemsets)

        0       1          2          3
0    Pain  Beurre  Confiture       None
1  Beurre    Coca       None       None
2  Beurre    Lait       None       None
3    Pain  Beurre       Coca       None
4    Pain    Lait       None       None
5  Beurre    Lait       None       None
6    Pain    Lait       None       None
7    Pain  Beurre       Lait  Confiture
8    Pain  Beurre       Lait       None


In [7]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary.astype(int), columns=te.columns_)
print(df)

   Beurre  Coca  Confiture  Lait  Pain
0       1     0          1     0     1
1       1     1          0     0     0
2       1     0          0     1     0
3       1     1          0     0     1
4       0     0          0     1     1
5       1     0          0     1     0
6       0     0          0     1     1
7       1     0          1     1     1
8       1     0          0     1     1


In [8]:
# Finding frequent itemsets with Eclat
min_support = 2/len(df)
frequent_itemsets = apriori(df.astype(bool), min_support=min_support, use_colnames=True)
print(frequent_itemsets)

     support                   itemsets
0   0.777778                   (Beurre)
1   0.222222                     (Coca)
2   0.222222                (Confiture)
3   0.666667                     (Lait)
4   0.666667                     (Pain)
5   0.222222             (Coca, Beurre)
6   0.222222        (Confiture, Beurre)
7   0.444444             (Lait, Beurre)
8   0.444444             (Pain, Beurre)
9   0.222222          (Confiture, Pain)
10  0.444444               (Lait, Pain)
11  0.222222  (Pain, Confiture, Beurre)
12  0.222222       (Lait, Pain, Beurre)


In [9]:
# Filtering for frequent itemsets with three items
frequent_3_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) == len(max(frequent_itemsets['itemsets'], key=len)))]
# Displaying the results
print(frequent_3_itemsets)

     support                   itemsets
11  0.222222  (Pain, Confiture, Beurre)
12  0.222222       (Lait, Pain, Beurre)


In [22]:
# Now, let's generate the association rules
#rules = association_rules(frequent_3_itemsets, metric="confidence", min_threshold=0.2,support_only=True)
#rules['confidence'] = rules['support'] / rules['antecedent support']

In [23]:
#print(rules[['antecedents', 'consequents', 'support', 'confidence']])

In [10]:
min_confidence = 0.7
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
rules_3_itemsets = rules[((rules['antecedents'].apply(len)) + (rules['consequents'].apply(len)) == len(max(frequent_itemsets['itemsets'], key=len)))]
rules_3_itemsets

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3,"(Confiture, Pain)",(Beurre),0.222222,0.777778,0.222222,1.0,1.285714,0.049383,inf,0.285714
4,"(Confiture, Beurre)",(Pain),0.222222,0.666667,0.222222,1.0,1.500000,0.074074,inf,0.428571
5,(Confiture),"(Beurre, Pain)",0.222222,0.444444,0.222222,1.0,2.250000,0.123457,inf,0.714286
